In [1]:
import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings("ignore")

prices=pd.read_csv("stock_prices.csv")
check=pd.read_csv("index_level_results_rounded.csv")

In [2]:
#Adding column for datetime format
dates=[]
for date in prices["Date"]:
    dates+=[datetime.datetime.strptime(date, "%d/%m/%Y")]
prices["datetime"]=dates

In [3]:
#Creating function to return names of Top 3 stocks
def get_top3(list):
    t_list=list
    idx=[]
    for i in range(3):
        m=max(t_list)
        i=list.index(m)
        idx+=[i+1]
        list[i]=0
        
    tickers= prices.columns[idx].to_list()
    return tickers

In [4]:
prices["index"]=0
the_three=get_top3(prices.iloc[0,1:-2].to_list())

for i in range(len(prices["Date"])):
    try:
        current_month=prices["datetime"][i].month
        next_month=prices["datetime"][i+1].month
    except:
        pass
    
    #Giving weights 0.5,0.25,0.25
    prices["index"][i]= prices[the_three].iloc[i][0]*0.5 + prices[the_three].iloc[i][1]*0.25 + prices[the_three].iloc[i][2]*0.25
    
    #Updating the Top 3 stocks for next month
    if not(next_month==current_month):
        the_three=get_top3(prices.iloc[i,1:-2].to_list())

In [5]:
prices["Final_Index"]=100
prices["Refrence_Values"]=100

#Adjusting the calculated index values
for i in range(3, len(prices["Date"])):
    prices["Final_Index"].iloc[i]= prices["Final_Index"].iloc[i-1] * (1 + ((prices["index"].iloc[i] / prices["index"].iloc[i-1])-1))

#Adding the given index values as a reference
for i in range(2, len(prices["Date"])):
    prices["Refrence_Values"].iloc[i]=check["index_level"].iloc[i-2]
    
# prices["Final_Index"] = [round(i,2) for i in prices["Final_Index"].to_list()]

#Removing  unnecessary columns
prices.drop(["index","datetime"], axis=1, inplace=True)
prices.head()

,Date,Stock_A,Stock_B,Stock_C,Stock_D,Stock_E,Stock_F,Stock_G,Stock_H,Stock_I,Stock_J,Final_Index,Refrence_Values
0,30/12/2019,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.000000,100.00
1,31/12/2019,99.35,101.10,100.55,99.66,100.15,99.50,100.33,100.39,99.99,99.95,100.000000,100.00
2,01/01/2020,99.85,100.51,100.12,98.09,100.10,100.41,103.67,101.16,100.00,101.22,100.000000,100.00
3,02/01/2020,101.12,101.67,101.23,98.25,99.94,100.96,103.99,100.99,100.20,102.34,100.810341,100.81
4,03/01/2020,100.65,101.61,102.31,99.62,100.77,102.29,103.24,101.64,100.90,101.98,101.210539,101.21
